# Pytorch Tutorial

### 4. Saving and Loading Models and Their States

- Saving and loading model parameters
- Using ```torchvision.models```

Setup torch and torchvision.

In [ ]:
import torch, torchvision
import torch.nn as nn
import torch.optim as optim

## Using ```torchvision.models```

Models provided by ```torchvision.models``` are:
- AlexNet
- VGG
- ResNet
- SqueezeNet
- DenseNet
- Inception v3

Refer to the [torchvision documentation](https://pytorch.org/docs/stable/torchvision/models.html) for details.

Here we load the AlexNet model structure.

In [ ]:
alex = torchvision.models.alexnet()
alex

Model parameters are initialized randomly.

In [ ]:
alex.state_dict()

We can also load pretrained models with the optional argument ```pretrained=True```.

In [ ]:
alex_pretrained = torchvision.models.alexnet(pretrained=True)

In [ ]:
alex_pretrained.state_dict()

Finetuning pretrained networks can be done by modifying the ```requires_grad``` parameter.

Here we switch on the gradients only for the last fully-connected layer.

In [ ]:
for param in alex_pretrained.parameters():
    param.requires_grad = False

# Parameters of newly created modules have requires_grad=True by default
alex_pretrained.classifier[6] = nn.Linear(in_features=4096, out_features=1000, bias=True)

In [ ]:
for name, param in alex_pretrained.named_parameters():
    print(f'{name: <20}: {param.requires_grad}')

## Saving and Loading Parameters

Saving the current parameters is implemented internally using pickle.

There are two ways:

1. **Saving the entire model**

    Here we save the model instance itself, including model structure and parameter values. Suitable for later inference after training is done. May be unstable or even break on future releases or in other projects.
  

2. **Saving only the parameter values** (recommended)

    Suitable for resuming training later. You may also save the optimizer state. Also ```state_dict``` is easier to manipulate and load into other models.

In [ ]:
model = torchvision.models.alexnet(pretrained=True)
optimizer = optim.Adam(model.parameters(), lr=5e-3)

In [ ]:
# Method 1: Saving the entire model
PATH = 'alexnet_pretrained_1'
torch.save(model, PATH)

In [ ]:
replica = torch.load(PATH)
print(type(replica))

In [ ]:
# Method 2: Saving only the parameters
PATH = 'alexnet_pretrained_2'

state = {
    'epoch': 30,
    'state_dict': model.state_dict(),
    'optimizer': optimizer.state_dict()
}

torch.save(state, PATH)

Load back states to a fresh instance of ```AlexNet``` and ```optim.Adam```.

In [ ]:
model = torchvision.models.alexnet()
optimizer = optim.Adam(model.parameters())

In [ ]:
old_state = torch.load(PATH)
model.load_state_dict(old_state['state_dict'])
optimizer.load_state_dict(old_state['optimizer'])

In [ ]:
optimizer.state_dict()['param_groups']